In [44]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import sys
import os

import matplotlib.pyplot as plt
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from random import randrange
import pandas as pd

import boto3

In [3]:
pwd

'/root/emotional-recognition/notebooks/low_level/supervised_learning/audio'

In [4]:
os.chdir('/root/emotional-recognition/notebooks/low_level/supervised_learning/audio/')

In [5]:
pwd

'/root/emotional-recognition/notebooks/low_level/supervised_learning/audio'

# GEMEP with egemaps

In [4]:
from s3fs.core import S3FileSystem
s3 = S3FileSystem()
from sagemaker import get_execution_role

role = get_execution_role()
bucket='files-and-examples-01'
file = 'datasets/gemep/gemep_audio_time_series.npz'

path = s3.open('s3://{}/{}'.format(bucket, file))

f = np.load(path, 'r')
x = f['x']
y = f['y']

In [5]:
x.shape

(1260, 842, 25)

In [6]:
len(np.unique(y))

18

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [15]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

In [16]:
seq_len = x.shape[1]
n_cols = x.shape[2]
output_dim = len(np.unique(y))

model = keras.Sequential(
    [
        layers.Masking(mask_value = -1000, input_shape=(seq_len, n_cols)),
        layers.LSTM(units=512,
                   return_sequences=True,
                   input_shape=(seq_len, n_cols)),
        layers.LSTM(units=256,
                   return_sequences=True,),        
        layers.Flatten(),
        layers.Dense(256, activation="sigmoid"),
        layers.Dense(128, activation="sigmoid"),
        layers.Dense(output_dim, activation="sigmoid")
    ]
)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              optimizer=optimizer, 
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_3 (Masking)          (None, 842, 25)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 842, 512)          1101824   
_________________________________________________________________
lstm_7 (LSTM)                (None, 842, 256)          787456    
_________________________________________________________________
flatten_3 (Flatten)          (None, 215552)            0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               55181568  
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 18)               

In [17]:
model.fit(X_train, y_train, batch_size=100, epochs=200, validation_data=(X_test, y_test), callbacks=[callback])

Epoch 1/200
9/9 [==============================] - 9s 435ms/step - loss: 2.9143 - accuracy: 0.0862 - val_loss: 2.8277 - val_accuracy: 0.1217
Epoch 2/200
9/9 [==============================] - 2s 191ms/step - loss: 2.7844 - accuracy: 0.1020 - val_loss: 2.7664 - val_accuracy: 0.1349
Epoch 3/200
9/9 [==============================] - 2s 192ms/step - loss: 2.7054 - accuracy: 0.1542 - val_loss: 2.7171 - val_accuracy: 0.1561
Epoch 4/200
9/9 [==============================] - 2s 193ms/step - loss: 2.6435 - accuracy: 0.1837 - val_loss: 2.6849 - val_accuracy: 0.1561
Epoch 5/200
9/9 [==============================] - 2s 193ms/step - loss: 2.5743 - accuracy: 0.2109 - val_loss: 2.6474 - val_accuracy: 0.1667
Epoch 6/200
9/9 [==============================] - 2s 189ms/step - loss: 2.5070 - accuracy: 0.2381 - val_loss: 2.6263 - val_accuracy: 0.1984
Epoch 7/200
9/9 [==============================] - 2s 196ms/step - loss: 2.4289 - accuracy: 0.3118 - val_loss: 2.6038 - val_accuracy: 0.2116
Epoch 8/200
9

In [20]:
y_pred = model.predict(X_test)

In [21]:
y_pred.shape

(378, 18)

In [22]:
y_pred[0]

array([0.22812739, 0.75123197, 0.26695713, 0.71414995, 0.30736938,
       0.80332845, 0.6166555 , 0.16934572, 0.3400268 , 0.5842783 ,
       0.27325326, 0.81106746, 0.3604107 , 0.9201705 , 0.30282494,
       0.492423  , 0.41047612, 0.31283128], dtype=float32)

# Confusion Matrix

In [25]:
layer = tf.keras.layers.Softmax()

1.0

In [45]:
# confusion matrix
unique_labels = np.unique(y)

ret = np.zeros((output_dim, output_dim))

for idx, pred in enumerate(y_pred):
    y_true = y_test[idx]
    ret[y_true] += pred
    
softmax_ret = layer(ret).numpy()

# Preicion and recall

In [72]:
gemep_emotion_abr_to_emotion = {'adm': 'admiration',
                                'amu': 'amusement',
                                'att': 'tenderness',
                                'col': 'anger',
                                'deg': 'disgust',
                                'des': 'despair',
                                'fie': 'pride',
                                'hon': 'shame',
                                'inq': 'anxiety',
                                'int': 'interest',
                                'irr': 'irritation',
                                'joi': 'joy',
                                'mep': 'contempt',
                                'peu': 'panic',
                                'pla': 'pleasure',
                                'sou': 'relief',
                                'sur': 'surprise',
                                'tri': 'sadness'
                                }

labels = list(gemep_emotion_abr_to_emotion.values())

In [73]:
labels

['admiration',
 'amusement',
 'tenderness',
 'anger',
 'disgust',
 'despair',
 'pride',
 'shame',
 'anxiety',
 'interest',
 'irritation',
 'joy',
 'contempt',
 'panic',
 'pleasure',
 'relief',
 'surprise',
 'sadness']

In [47]:
softmax_y_pred = layer(y_pred).numpy()

In [55]:
y_pred_categorical = np.argmax(softmax_y_pred, axis=1)

In [76]:
np.unique(y_pred_categorical)

array([ 1,  3,  5,  6,  8,  9, 10, 11, 13, 14, 15, 17])

In [75]:
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred_categorical, target_names=labels, digits=3))

              precision    recall  f1-score   support

  admiration      0.000     0.000     0.000         8
   amusement      0.269     0.241     0.255        29
  tenderness      0.000     0.000     0.000         9
       anger      0.217     0.152     0.179        33
     disgust      0.000     0.000     0.000        10
     despair      0.156     0.238     0.189        21
       pride      0.163     0.258     0.200        31
       shame      0.000     0.000     0.000         8
     anxiety      0.194     0.286     0.231        21
    interest      0.057     0.069     0.062        29
  irritation      0.211     0.182     0.195        22
         joy      0.333     0.259     0.292        27
    contempt      0.000     0.000     0.000         8
       panic      0.395     0.484     0.435        31
    pleasure      0.333     0.229     0.271        35
      relief      0.192     0.185     0.189        27
    surprise      0.000     0.000     0.000         7
     sadness      0.185    

/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
from sklearn.metrics import recall_score

print(recall_score(y_test, y_pred_categorical, average='macro'))
print(recall_score(y_test, y_pred_categorical, average='micro'))
print(recall_score(y_test, y_pred_categorical, average='weighted'))

0.1687213608978533
0.21693121693121692
0.21693121693121692


# GEMEP with Compare

In [6]:
from s3fs.core import S3FileSystem
s3 = S3FileSystem()
from sagemaker import get_execution_role

role = get_execution_role()
bucket='files-and-examples-01'
file = 'datasets/gemep/gemep_compare_audio_time_series.npz'

path = s3.open('s3://{}/{}'.format(bucket, file))

f = np.load(path, 'r')
x = f['x']
y = f['y']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [25]:
seq_len = x.shape[1]
n_cols = x.shape[2]
output_dim = len(np.unique(y))

model = keras.Sequential(
    [
        layers.Masking(mask_value = -1000, input_shape=(seq_len, n_cols)),
        layers.LSTM(units=512,
                   return_sequences=True,
                   input_shape=(seq_len, n_cols)),
        layers.LSTM(units=256,
                   return_sequences=True,),        
        layers.Flatten(),
        layers.Dense(256, activation="sigmoid"),
        layers.Dense(128, activation="sigmoid"),
        layers.Dense(output_dim, activation="sigmoid")
    ]
)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              optimizer=optimizer, 
              metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_5 (Masking)          (None, 842, 65)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 842, 512)          1183744   
_________________________________________________________________
lstm_11 (LSTM)               (None, 842, 256)          787456    
_________________________________________________________________
flatten_5 (Flatten)          (None, 215552)            0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               55181568  
_________________________________________________________________
dense_16 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_17 (Dense)             (None, 18)               

In [26]:
model.fit(X_train, y_train, batch_size=100, epochs=200, validation_data=(X_test, y_test))

Epoch 1/200
9/9 [==============================] - 10s 458ms/step - loss: 2.9154 - accuracy: 0.0771 - val_loss: 2.8092 - val_accuracy: 0.1032
Epoch 2/200
9/9 [==============================] - 2s 195ms/step - loss: 2.8002 - accuracy: 0.0975 - val_loss: 2.7934 - val_accuracy: 0.1190
Epoch 3/200
9/9 [==============================] - 2s 199ms/step - loss: 2.7708 - accuracy: 0.1168 - val_loss: 2.7721 - val_accuracy: 0.1243
Epoch 4/200
9/9 [==============================] - 2s 194ms/step - loss: 2.7385 - accuracy: 0.1429 - val_loss: 2.7535 - val_accuracy: 0.1190
Epoch 5/200
9/9 [==============================] - 2s 201ms/step - loss: 2.7030 - accuracy: 0.1587 - val_loss: 2.7361 - val_accuracy: 0.1349
Epoch 6/200
9/9 [==============================] - 2s 196ms/step - loss: 2.6675 - accuracy: 0.1803 - val_loss: 2.7116 - val_accuracy: 0.1376
Epoch 7/200
9/9 [==============================] - 2s 200ms/step - loss: 2.6230 - accuracy: 0.2063 - val_loss: 2.6906 - val_accuracy: 0.1455
Epoch 8/200


KeyboardInterrupt: 